#Entrenamiento del modelo MentalRoBERTa

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━

You will need to setup git, adapt your email and name in the following cell.

In [ ]:
!git config --global user.email "luisangel.motta@alumnos.viu.es"
!git config --global user.name "AngelMV97"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset
# Cargue del dataset
raw_datasets = load_dataset("LuangMV97/EmpathetiCounseling_Dataset")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/47998 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12002 [00:00<?, ? examples/s]

In [ ]:
for key in raw_datasets["train"][0]:
    print(f"{key.upper()}: {raw_datasets['train'][0][key]}")

INPUT: I will have to look at that.  I've not been in the job market for 25 years.  I thought they may be able to provide some assistance in just where do I begin. 4
I'm a teacher at a private school.  4
LABEL: You want to know where to start.  Sounds like you are young.


In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling

context_length = 512
checkpoint = "mental/mental-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForMaskedLM.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/327 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/682 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
list_input=list(raw_datasets['train']['input'])
list_label=list(raw_datasets['train']['label'])
max_length_input=len(max(list_input, key=len))
max_length_label=len(max(list_label, key=len))
print(f'Longitud máxima (hablante): {max_length_input}\nLongitud máxima (oyente): {max_length_label}')

Longitud máxima (hablante): 2702
Longitud máxima (oyente): 87747


In [ ]:
def tokenize_function(example):
    return tokenizer(text=example["input"],
                     text_target=example["label"],
                     truncation=True,
                     padding='max_length',
                     max_length=context_length,
                     return_length=True)
# Tokenización del conjunto de datos
tokenized_datasets = raw_datasets.map(tokenize_function,
                                      batched=True,
                                      remove_columns=raw_datasets["train"].column_names)

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=True)

Map:   0%|          | 0/47998 [00:00<?, ? examples/s]

Map:   0%|          | 0/12002 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'length', 'labels'],
        num_rows: 47998
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'length', 'labels'],
        num_rows: 12002
    })
})

In [ ]:
out = data_collator([tokenized_datasets["train"][i] for i in range(5)])
for key in out:
    print(f"{key} shape: {out[key].shape}")

input_ids shape: torch.Size([5, 512])
attention_mask shape: torch.Size([5, 512])
length shape: torch.Size([5])
labels shape: torch.Size([5, 512])


Instancia el Entrenador

In [ ]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="MentalRoBERTa_EmpAI_final3",
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=1_000,
    auto_find_batch_size=True,
    gradient_accumulation_steps=8,
    max_steps=5_000,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.1,
    learning_rate=1e-4,
    warmup_steps=10_000,
    fp16=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Pruebas de entrenamiento del modelo

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss
1000,3.442100,0.448880


Step,Training Loss,Validation Loss
1000,3.442100,0.448880
2000,0.076600,0.072433
3000,0.070100,0.069115
4000,0.065800,0.067273
5000,0.063200,0.070058


TrainOutput(global_step=5000, training_loss=1.339535676574707, metrics={'train_runtime': 6907.5594, 'train_samples_per_second': 46.326, 'train_steps_per_second': 0.724, 'total_flos': 8.895950617142477e+16, 'train_loss': 1.339535676574707, 'epoch': 6.67})

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 1.07


In [ ]:
eval_results

{'eval_loss': 0.06888021528720856,
 'eval_runtime': 79.8285,
 'eval_samples_per_second': 150.347,
 'eval_steps_per_second': 9.408,
 'epoch': 6.67}

In [ ]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/LuangMV97/MentalRoBERTa_EmpAI_final3/commit/d140d60fc9fa1af11ba5f839048ab01ec0456251', commit_message='End of training', commit_description='', oid='d140d60fc9fa1af11ba5f839048ab01ec0456251', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.train() #weight_decay=0.1, learning_rate=1e-4, warmup_sterps=10_000 MentalRoBERTa_EmpAI_final

Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss
1000,3.724300,0.516079
2000,0.075700,0.071648


Step,Training Loss,Validation Loss
1000,3.724300,0.516079
2000,0.075700,0.071648
3000,0.070200,0.070279


TrainOutput(global_step=3750, training_loss=2.051880960083008, metrics={'train_runtime': 4567.4501, 'train_samples_per_second': 52.544, 'train_steps_per_second': 0.821, 'total_flos': 6.326111123724902e+16, 'train_loss': 2.051880960083008, 'epoch': 5.0})

In [ ]:
trainer.train() #weight_decay=0.1, learning_rate=1e-4, warmup_sterps=10_000 RoBERTa_EmpAI_final

Epoch,Training Loss,Validation Loss
1,12.692100,4.906155
2,0.377700,0.080903
3,0.077900,0.071495
4,0.070400,0.069692
5,0.068800,0.070380


TrainOutput(global_step=3750, training_loss=2.4424086278279624, metrics={'train_runtime': 4680.6633, 'train_samples_per_second': 51.273, 'train_steps_per_second': 0.801, 'total_flos': 6.318107853640704e+16, 'train_loss': 2.4424086278279624, 'epoch': 5.0})

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Step,Training Loss,Validation Loss
681,0.060200,0.066708


Perplexity: 1.07


In [ ]:
eval_results

{'eval_loss': 0.06670784205198288}

In [ ]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

events.out.tfevents.1712325383.deabfdb04fe4.818.4:   0%|          | 0.00/9.73k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/LuangMV97/MentalRoBERTa_EmpAI_final/commit/935abff5d0deacd8e45f4af3f9a86c1fe1339bcb', commit_message='End of training', commit_description='', oid='935abff5d0deacd8e45f4af3f9a86c1fe1339bcb', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.train() #weight_decay=0.1, learning_rate=1e-5, warmup_sterps=500 RoBERTa_EmpTalk_Prueba

Epoch,Training Loss,Validation Loss
0,No log,7.425834
2,4.901900,0.087168
4,0.403600,0.076457


Epoch,Training Loss,Validation Loss
0,No log,7.425834
2,4.901900,0.087168
4,0.080200,0.075440


TrainOutput(global_step=2415, training_loss=3.7459706292636152, metrics={'train_runtime': 3051.9735, 'train_samples_per_second': 50.684, 'train_steps_per_second': 0.791, 'total_flos': 4.068293857405747e+16, 'train_loss': 3.7459706292636152, 'epoch': 4.99})

In [ ]:
trainer.train(resume_from_checkpoint=True)

There were missing keys in the checkpoint model loaded: ['lm_head.decoder.weight', 'lm_head.decoder.bias'].


Epoch,Training Loss,Validation Loss
5,0.073700,0.070847
7,0.071200,0.070178
9,0.069800,0.069570


TrainOutput(global_step=4830, training_loss=0.0357159656027089, metrics={'train_runtime': 3046.3138, 'train_samples_per_second': 101.556, 'train_steps_per_second': 1.586, 'total_flos': 8.136587714811494e+16, 'train_loss': 0.0357159656027089, 'epoch': 9.99})

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 1.07


In [ ]:
eval_results

{'eval_loss': 0.07068553566932678,
 'eval_runtime': 56.2572,
 'eval_samples_per_second': 137.511,
 'eval_steps_per_second': 17.189,
 'epoch': 9.99}

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1712048727.cddba71158cf.1161.3:   0%|          | 0.00/359 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/LuangMV97/RoBERTa_EmpAI_FineTuned/commit/65155065b9408af51282a401333da82d406bdba7', commit_message='End of training', commit_description='', oid='65155065b9408af51282a401333da82d406bdba7', pr_url=None, pr_revision=None, pr_num=None)